In [58]:
from torchvision.models.mobilenetv3 import mobilenet_v3_small
from torchvision.models import efficientnet, AlexNet
from collections import OrderedDict
from torchinfo import summary

In [46]:

def set_parameters(model,parameters):
    key = [k for k in model.state_dict().keys() if 'bn' not in k]
    params_dict = zip(key, parameters)
    state_dict = OrderedDict({k: v for k, v in params_dict})
    print(state_dict.keys())
    model.load_state_dict(state_dict, strict=False)
    
def get_parameters(model):
    return [val.cpu() for name,val in model.state_dict().items() if 'bn' not in name]


In [65]:
import torch
x = torch.randn(3, 3, 224, 224)
efficient(x).shape

torch.Size([3, 10])

In [63]:
model = AlexNet(num_classes=10)
efficient = efficientnet.efficientnet_b0(num_classes=10)
summary(efficient)

Layer (type:depth-idx)                                  Param #
EfficientNet                                            --
├─Sequential: 1-1                                       --
│    └─Conv2dNormActivation: 2-1                        --
│    │    └─Conv2d: 3-1                                 864
│    │    └─BatchNorm2d: 3-2                            64
│    │    └─SiLU: 3-3                                   --
│    └─Sequential: 2-2                                  --
│    │    └─MBConv: 3-4                                 1,448
│    └─Sequential: 2-3                                  --
│    │    └─MBConv: 3-5                                 6,004
│    │    └─MBConv: 3-6                                 10,710
│    └─Sequential: 2-4                                  --
│    │    └─MBConv: 3-7                                 15,350
│    │    └─MBConv: 3-8                                 31,290
│    └─Sequential: 2-5                                  --
│    │    └─MBConv: 3-9         

In [84]:
ok = list(efficient.state_dict().keys())
ok

['features.0.0.weight',
 'features.0.1.weight',
 'features.0.1.bias',
 'features.0.1.running_mean',
 'features.0.1.running_var',
 'features.0.1.num_batches_tracked',
 'features.1.0.block.0.0.weight',
 'features.1.0.block.0.1.weight',
 'features.1.0.block.0.1.bias',
 'features.1.0.block.0.1.running_mean',
 'features.1.0.block.0.1.running_var',
 'features.1.0.block.0.1.num_batches_tracked',
 'features.1.0.block.1.fc1.weight',
 'features.1.0.block.1.fc1.bias',
 'features.1.0.block.1.fc2.weight',
 'features.1.0.block.1.fc2.bias',
 'features.1.0.block.2.0.weight',
 'features.1.0.block.2.1.weight',
 'features.1.0.block.2.1.bias',
 'features.1.0.block.2.1.running_mean',
 'features.1.0.block.2.1.running_var',
 'features.1.0.block.2.1.num_batches_tracked',
 'features.2.0.block.0.0.weight',
 'features.2.0.block.0.1.weight',
 'features.2.0.block.0.1.bias',
 'features.2.0.block.0.1.running_mean',
 'features.2.0.block.0.1.running_var',
 'features.2.0.block.0.1.num_batches_tracked',
 'features.2.0.b

In [83]:
a = list(model1.state_dict().keys())
tensor1 = {k:v.size() for k,v in efficient.state_dict().items() if 'num_batches_tracked' in k}
tensor1

{'features.0.1.num_batches_tracked': torch.Size([]),
 'features.1.0.block.0.1.num_batches_tracked': torch.Size([]),
 'features.1.0.block.2.1.num_batches_tracked': torch.Size([]),
 'features.2.0.block.0.1.num_batches_tracked': torch.Size([]),
 'features.2.0.block.1.1.num_batches_tracked': torch.Size([]),
 'features.2.0.block.3.1.num_batches_tracked': torch.Size([]),
 'features.2.1.block.0.1.num_batches_tracked': torch.Size([]),
 'features.2.1.block.1.1.num_batches_tracked': torch.Size([]),
 'features.2.1.block.3.1.num_batches_tracked': torch.Size([]),
 'features.3.0.block.0.1.num_batches_tracked': torch.Size([]),
 'features.3.0.block.1.1.num_batches_tracked': torch.Size([]),
 'features.3.0.block.3.1.num_batches_tracked': torch.Size([]),
 'features.3.1.block.0.1.num_batches_tracked': torch.Size([]),
 'features.3.1.block.1.1.num_batches_tracked': torch.Size([]),
 'features.3.1.block.3.1.num_batches_tracked': torch.Size([]),
 'features.4.0.block.0.1.num_batches_tracked': torch.Size([]),
 '

In [48]:
params = get_parameters(model2)

In [49]:
set_parameters(model1,params)

odict_keys(['features.0.0.weight', 'features.0.1.weight', 'features.0.1.bias', 'features.0.1.running_mean', 'features.0.1.running_var', 'features.0.1.num_batches_tracked', 'features.1.block.0.0.weight', 'features.1.block.0.1.weight', 'features.1.block.0.1.bias', 'features.1.block.0.1.running_mean', 'features.1.block.0.1.running_var', 'features.1.block.0.1.num_batches_tracked', 'features.1.block.1.fc1.weight', 'features.1.block.1.fc1.bias', 'features.1.block.1.fc2.weight', 'features.1.block.1.fc2.bias', 'features.1.block.2.0.weight', 'features.1.block.2.1.weight', 'features.1.block.2.1.bias', 'features.1.block.2.1.running_mean', 'features.1.block.2.1.running_var', 'features.1.block.2.1.num_batches_tracked', 'features.2.block.0.0.weight', 'features.2.block.0.1.weight', 'features.2.block.0.1.bias', 'features.2.block.0.1.running_mean', 'features.2.block.0.1.running_var', 'features.2.block.0.1.num_batches_tracked', 'features.2.block.1.0.weight', 'features.2.block.1.1.weight', 'features.2.bl

In [26]:
model1 

MobileNetV3(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      (2): Hardswish()
    )
    (1): InvertedResidual(
      (block): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=16, bias=False)
          (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
        )
        (1): SqueezeExcitation(
          (avgpool): AdaptiveAvgPool2d(output_size=1)
          (fc1): Conv2d(16, 8, kernel_size=(1, 1), stride=(1, 1))
          (fc2): Conv2d(8, 16, kernel_size=(1, 1), stride=(1, 1))
          (activation): ReLU()
          (scale_activation): Hardsigmoid()
        )
        (2): Conv2dNormActivation(
          (0): Conv2d(16, 16, kernel_size=(1, 1), 